In [ ]:
!pip install -U \
  transformers==4.46.3 \
  accelerate==0.34.2 \
  peft==0.13.2 \
  trl==0.11.4

In [ ]:
import torch
import transformers
import accelerate
import peft
import trl

print("torch        :", torch.__version__)
print("transformers :", transformers.__version__)
print("accelerate   :", accelerate.__version__)
print("peft         :", peft.__version__)
print("trl          :", trl.__version__)

## 1. 환경 설정

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 라이브러리 import
import os
import torch
from pathlib import Path
from typing import Dict, Any
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)
from trl import SFTTrainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. 설정

In [ ]:
# ===== 설정 =====
from pathlib import Path

BASE_PATH = Path("/content/drive/MyDrive/Colab Notebooks/LikeLion/종합 프로젝트/demo-repository/lora")

# 데이터 경로 (기존과 동일)
DATA_PATH = BASE_PATH / "data" / "stepmother_dialogue_00.jsonl"

# 출력(LoRA 어댑터) 경로
OUTPUT_DIR = BASE_PATH / "adapters/stepmother_qwen_00_01"

# 모델 설정 (Qwen 8B로 변경)
MODEL_NAME = "Qwen/Qwen2.5-8B-Instruct"

# LoRA 설정
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
# Qwen 모델의 target modules
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# 학습 설정
NUM_EPOCHS = 3
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 8
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 512

# 출력 디렉토리 생성
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Data path: {DATA_PATH}")
print(f"Output dir: {OUTPUT_DIR}")

## 3. 양자화 및 LoRA 설정

In [ ]:
# 양자화 설정 (4-bit NF4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
print("Quantization config ready (4-bit NF4)")

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=TARGET_MODULES,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
print(f"LoRA config ready (r={LORA_R}, alpha={LORA_ALPHA})")

## 4. 모델 및 토크나이저 로드

In [ ]:
# 토크나이저 로드
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("Tokenizer loaded")

In [ ]:
# 모델 로드
print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
print("Model loaded")

In [ ]:
# LoRA는 SFTTrainer에서 자동 적용됨
print(f"LoRA will be applied by SFTTrainer (r={LORA_R}, alpha={LORA_ALPHA})")

## 5. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
dataset = load_dataset("json", data_files=str(DATA_PATH), split="train")
print(f"Loaded {len(dataset)} training samples")

# 샘플 확인
print("\n=== Sample Data ===")
print(dataset[0])

In [ ]:
# 데이터 포맷 함수 (Qwen ChatML format)
def format_instruction(samples: Dict[str, Any]) -> list[str]:
    """
    학습 데이터를 Qwen ChatML format으로 변환 (배치 처리)

    Qwen ChatML format:
    <|im_start|>system
    시스템 메시지<|im_end|>
    <|im_start|>user
    사용자 입력<|im_end|>
    <|im_start|>assistant
    어시스턴트 응답<|im_end|>
    """
    system_msg = (
        "당신은 집착적이고 왜곡된 보호 본능을 가진 몬스터입니다. "
        "문장은 반복되거나 늘어질 수 있으며 감정이 급격히 전환됩니다. "
        "의성어나 의태어는 사용하지 않습니다."
    )

    results = []
    for user_msg, output in zip(samples["input"], samples["output"]):
        formatted = (
            f"<|im_start|>system\n{system_msg}<|im_end|>\n"
            f"<|im_start|>user\n{user_msg}<|im_end|>\n"
            f"<|im_start|>assistant\n{output}<|im_end|>"
        )
        results.append(formatted)

    return results

# 포맷 테스트
print("\n=== Formatted Sample ===")
print(format_instruction({"input": [dataset[0]["input"]], "output": [dataset[0]["output"]]}))

## 6. 학습 설정

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    group_by_length=True,
    report_to="none",
)

print("Training arguments configured")

In [ ]:
# SFTTrainer 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    peft_config=lora_config,
    formatting_func=format_instruction,
    max_seq_length=MAX_SEQ_LENGTH,
)

print("Trainer ready")

## 7. 학습 실행

In [ ]:
print("=" * 60)
print("Monster Style LoRA Training for Qwen 2.5 8B")
print("=" * 60)
print("\n목적: 몬스터 말투(문장 구조, 반복, 붕괴된 문법, 광기 표현)만 학습")
print("주의: 게임 로직, humanity 변수, 상태 전이, semantic role은 포함하지 않음\n")
print("Starting training...")
print("=" * 60 + "\n")

# 학습 시작
trainer.train()

## 8. 모델 저장

In [ ]:
# LoRA 어댑터 저장
print(f"Saving model to: {OUTPUT_DIR}")
trainer.model.save_pretrained(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))

print("\n" + "=" * 60)
print("Training completed!")
print("=" * 60)

## 9. 추론 테스트

In [ ]:
# 추론 테스트 함수
def test_inference(prompt: str) -> str:
    """학습된 LoRA로 추론 테스트"""
    system_msg = "당신은 집착적이고 왜곡된 보호 본능을 가진 몬스터입니다. 문장은 반복되거나 늘어질 수 있으며 감정이 급격히 전환됩니다."
    formatted_prompt = (
        f"<|im_start|>system\n{system_msg}<|im_end|>\n"
        f"<|im_start|>user\n{prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response.split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "").strip()

    return response

In [ ]:
# 테스트 실행
test_prompts = [
    "안녕하세요, 만나서 반갑습니다.",
    "몬스터처럼 말해줘.",
    "다음 문장을 몬스터처럼 말해줘.\n\n배가 고파요.",
]

print("=== Inference Test ===")
for prompt in test_prompts:
    response = test_inference(prompt)
    print(f"\nInput: {prompt}")
    print(f"Monster Response: {response}")
    print("-" * 50)

## 10. 저장된 어댑터 로드 (별도 세션에서 사용시)

In [ ]:
# 이 셀은 별도 세션에서 학습된 어댑터를 로드할 때 사용
# 주석 해제 후 실행

'''
from peft import PeftModel

ADAPTER_PATH = "/content/drive/MyDrive/Colab Notebooks/LikeLion/종합 프로젝트/demo-repository/lora/adapters/stepmother_qwen_00_01"
MODEL_NAME = "Qwen/Qwen2.5-8B-Instruct"

# 기본 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# LoRA 어댑터 로드
model = PeftModel.from_pretrained(model, ADAPTER_PATH)
model.eval()

print("Adapter loaded successfully!")
'''